In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import logging
from datetime import datetime

# Build the Spark Session using AWS Keys from Databricks Secrets
spark = SparkSession.builder \
    .appName("PsychoBunny-DataIngestion") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key", dbutils.secrets.get(scope="aws-keys", key="aws-access-key")) \
    .config("spark.hadoop.fs.s3a.secret.key", dbutils.secrets.get(scope="aws-keys", key="aws-secret-key")) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Spark session initialized with Delta Lake and AWS S3 support")


INFO:__main__:Spark session initialized with Delta Lake and AWS S3 support
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


In [0]:
access = dbutils.secrets.get("aws-keys", key="aws-access-key")
secret = dbutils.secrets.get("aws-keys", key="aws-secret-key")

spark.conf.set("fs.s3a.access.key", access)
spark.conf.set("fs.s3a.secret.key", secret)
spark.conf.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")    

In [0]:
# Data paths
RAW_DATA_PATH = "s3://psycho-bunny-data-lake/raw-data/"
PROCESSED_DATA_PATH = "s3://psycho-bunny-data-lake/processed-data/"

logger.info("Analytics Requirements notebook initialized")

# Load processed data from Delta Lake
try:
    fact_transactions = spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}fact_transactions")
    dim_customer = spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}dim_customer")
    dim_product = spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}dim_product")
    calendar_df = spark.read.format("delta").load(f"{RAW_DATA_PATH}calendar")
    
    logger.info(f"Loaded data: {fact_transactions.count()} transactions, {dim_customer.count()} customers")
    
except Exception as e:
    logger.error(f"Error loading data: {str(e)}")
    raise


INFO:__main__:✅ Analytics Requirements notebook initialized
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py

In [0]:
# 1. Weekly, Monthly, Quarterly Sales Analysis
sales_analytics = fact_transactions.filter(col("transaction_type") == "SALE") \
    .withColumn("week", weekofyear(col("order_date"))) \
    .withColumn("month", month(col("order_date"))) \
    .withColumn("quarter", quarter(col("order_date"))) \
    .withColumn("year", year(col("order_date"))) \
    .groupBy("year", "quarter", "month", "week") \
    .agg(
        sum("net_amount").alias("total_sales"),
        count("order_number").alias("total_orders"),
        avg("net_amount").alias("avg_order_value")
    ).withColumn("created_date", current_timestamp())

logger.info(f"Sales Analytics: {sales_analytics.count()} records")
sales_analytics.orderBy(desc("total_sales")).show(10)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Sales Analytics: 1 records
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+----+-------+-----+----+-----------+------------+---------------+--------------------+
|year|quarter|month|week|total_sales|total_orders|avg_order_value|        created_date|
+----+-------+-----+----+-----------+------------+---------------+--------------------+
|2025|      2|    6|  25|   282300.0|        2823|          100.0|2025-06-21 21:28:...|
+----+-------+-----+----+-----------+------------+---------------+--------------------+



In [0]:
# 2. Weekly, Monthly, Quarterly Refunds Analysis
refunds_analytics = fact_transactions.filter(col("transaction_type") == "REFUND") \
    .withColumn("week", weekofyear(col("order_date"))) \
    .withColumn("month", month(col("order_date"))) \
    .withColumn("quarter", quarter(col("order_date"))) \
    .withColumn("year", year(col("order_date"))) \
    .groupBy("year", "quarter", "month", "week") \
    .agg(
        sum("net_amount").alias("total_refunds"),
        sum("restocking_fee").alias("total_restocking_fees"),
        count("order_number").alias("total_refund_orders")
    ).withColumn("created_date", current_timestamp())

logger.info(f"Refunds Analytics: {refunds_analytics.count()} records")
refunds_analytics.show(10)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Refunds Analytics: 0 records


+----+-------+-----+----+-------------+---------------------+-------------------+------------+
|year|quarter|month|week|total_refunds|total_restocking_fees|total_refund_orders|created_date|
+----+-------+-----+----+-------------+---------------------+-------------------+------------+
+----+-------+-----+----+-------------+---------------------+-------------------+------------+



In [0]:
# 3. Product Family Analysis - Extract from product codes
def extract_product_family(product_code):
    """Extract product family from product code"""
    if product_code and "_" in product_code:
        return product_code.split("_")[0]
    return "UNKNOWN"

extract_family_udf = udf(extract_product_family, StringType())

# Product family sales analysis
product_family_sales = fact_transactions.filter(col("transaction_type") == "SALE") \
    .withColumn("product_family", extract_family_udf(col("product_code"))) \
    .groupBy("product_family") \
    .agg(
        sum("net_amount").alias("total_sales"),
        count("order_number").alias("total_orders"),
        avg("net_amount").alias("avg_order_value")
    ).orderBy(desc("total_sales")) \
    .withColumn("created_date", current_timestamp())

logger.info(f"Product Family Sales: {product_family_sales.count()} families")
product_family_sales.show(10)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Product Family Sales: 8 families
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+--------------+-----------+------------+---------------+--------------------+
|product_family|total_sales|total_orders|avg_order_value|        created_date|
+--------------+-----------+------------+---------------+--------------------+
|           S18|    99500.0|         995|          100.0|2025-06-21 21:29:...|
|           S24|    73100.0|         731|          100.0|2025-06-21 21:29:...|
|          S700|    31300.0|         313|          100.0|2025-06-21 21:29:...|
|           S12|    25900.0|         259|          100.0|2025-06-21 21:29:...|
|           S32|    20600.0|         206|          100.0|2025-06-21 21:29:...|
|           S10|    16100.0|         161|          100.0|2025-06-21 21:29:...|
|           S50|    10500.0|         105|          100.0|2025-06-21 21:29:...|
|           S72|     5300.0|          53|          100.0|2025-06-21 21:29:...|
+--------------+-----------+------------+---------------+--------------------+



In [0]:
from pyspark.sql.window import Window

# 4. Best-selling and Second Best-selling Items by Region
regional_product_sales = fact_transactions.filter(col("transaction_type") == "SALE") \
    .withColumn("product_family", extract_family_udf(col("product_code"))) \
    .groupBy("territory", "product_code", "product_family") \
    .agg(
        sum("net_amount").alias("total_sales"),
        count("order_number").alias("total_orders")
    )

# Window function to rank products by sales within each territory
window_spec = Window.partitionBy("territory").orderBy(desc("total_sales"))

regional_rankings = regional_product_sales.withColumn(
    "rank", row_number().over(window_spec)
).filter(col("rank") <= 2) \
.withColumn("ranking_type", 
    when(col("rank") == 1, "Best Selling")
    .when(col("rank") == 2, "Second Best Selling")
    .otherwise("Other")
).withColumn("created_date", current_timestamp())

logger.info(f"Regional Rankings: {regional_rankings.count()} records")
regional_rankings.show(20)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Regional Rankings: 6 records
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+---------+------------+--------------+-----------+------------+----+-------------------+--------------------+
|territory|product_code|product_family|total_sales|total_orders|rank|       ranking_type|        created_date|
+---------+------------+--------------+-----------+------------+----+-------------------+--------------------+
|    Large|    S10_1949|           S10|     1400.0|          14|   1|       Best Selling|2025-06-21 21:31:...|
|    Large|    S18_3232|           S18|     1400.0|          14|   2|Second Best Selling|2025-06-21 21:31:...|
|   Medium|    S18_3232|           S18|     3400.0|          34|   1|       Best Selling|2025-06-21 21:31:...|
|   Medium|    S18_1129|           S18|     2300.0|          23|   2|Second Best Selling|2025-06-21 21:31:...|
|    Small|    S24_1444|           S24|     2600.0|          26|   1|       Best Selling|2025-06-21 21:31:...|
|    Small|    S18_2432|           S18|     2500.0|          25|   2|Second Best Selling|2025-06-21 21:31:...|
+

In [0]:
# 5. Revenue Difference Between Best and Second Best Items per Region
revenue_difference = regional_rankings.groupBy("territory") \
    .agg(
        max(when(col("rank") == 1, col("total_sales"))).alias("best_selling_revenue"),
        max(when(col("rank") == 2, col("total_sales"))).alias("second_best_revenue")
    ).withColumn(
        "revenue_difference", 
        col("best_selling_revenue") - col("second_best_revenue")
    ).withColumn(
        "revenue_difference_pct",
        round((col("revenue_difference") / col("best_selling_revenue")) * 100, 2)
    ).withColumn("created_date", current_timestamp())

logger.info(f"Revenue Difference Analysis: {revenue_difference.count()} territories")
revenue_difference.show(10)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Revenue Difference Analysis: 3 territories
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+---------+--------------------+-------------------+------------------+----------------------+--------------------+
|territory|best_selling_revenue|second_best_revenue|revenue_difference|revenue_difference_pct|        created_date|
+---------+--------------------+-------------------+------------------+----------------------+--------------------+
|    Large|              1400.0|             1400.0|               0.0|                   0.0|2025-06-21 21:32:...|
|   Medium|              3400.0|             2300.0|            1100.0|                 32.35|2025-06-21 21:32:...|
|    Small|              2600.0|             2500.0|             100.0|                  3.85|2025-06-21 21:32:...|
+---------+--------------------+-------------------+------------------+----------------------+--------------------+



In [0]:
# 6. Enhanced Customer Segmentation (High, Medium, Low Value)
customer_metrics = fact_transactions.filter(col("transaction_type") == "SALE") \
    .groupBy("customer_name") \
    .agg(
        sum("net_amount").alias("total_spent"),
        count("order_number").alias("total_orders"),
        avg("net_amount").alias("avg_order_value"),
        max("order_date").alias("last_order_date"),
        countDistinct("product_code").alias("unique_products_purchased")
    )

# Calculate percentiles for segmentation
percentiles = customer_metrics.select(
    expr("percentile_approx(total_spent, 0.33)").alias("p33"),
    expr("percentile_approx(total_spent, 0.67)").alias("p67")
).collect()[0]

p33_threshold = percentiles["p33"]
p67_threshold = percentiles["p67"]

enhanced_customer_segments = customer_metrics.withColumn(
    "customer_segment",
    when(col("total_spent") >= p67_threshold, "High Value")
    .when(col("total_spent") >= p33_threshold, "Medium Value")
    .otherwise("Low Value")
).withColumn("created_date", current_timestamp())

logger.info(f"Enhanced Customer Segments: {enhanced_customer_segments.count()} customers")
logger.info(f"Thresholds: Low: <${p33_threshold:.2f}, Medium: ${p33_threshold:.2f}-${p67_threshold:.2f}, High: >${p67_threshold:.2f}")
enhanced_customer_segments.groupBy("customer_segment").count().show()


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Enhanced Customer Segments: 1 customers
INFO:__main__:Thresholds: Low: <$282300.00, Medium: $282300.00-$282300.00, High: >$282300.00
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+----------------+-----+
|customer_segment|count|
+----------------+-----+
|      High Value|    1|
+----------------+-----+



In [0]:
# 7. Top 10 Customers by Segment with Contact Details
top_customers_by_segment = enhanced_customer_segments.join(
    dim_customer.select("full_name", "email", "phone", "city", "state"),
    enhanced_customer_segments.customer_name == dim_customer.full_name,
    "left"
).select(
    "customer_name", "email", "phone", "city", "state",
    "total_spent", "total_orders", "avg_order_value", 
    "unique_products_purchased", "customer_segment"
).orderBy(desc("total_spent"))

logger.info(f"Top Customers with Contact Details: {top_customers_by_segment.count()} customers")
top_customers_by_segment.show(10)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Top Customers with Contact Details: 1 customers
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+-------------+--------------------+------------+---------+-----+-----------+------------+---------------+-------------------------+----------------+
|customer_name|               email|       phone|     city|state|total_spent|total_orders|avg_order_value|unique_products_purchased|customer_segment|
+-------------+--------------------+------------+---------+-----+-----------+------------+---------------+-------------------------+----------------+
| Aaron Kloska|aaron_kloska@klos...|07-9896-4827|Brookhill|  QLD|   282300.0|        2823|          100.0|                      109|      High Value|
+-------------+--------------------+------------+---------+-----+-----------+------------+---------------+-------------------------+----------------+



In [0]:
# 8. Refund UDF with 10% Restocking Fee
def calculate_refund_with_fee(original_amount, is_refund):
    """Calculate final refund amount with 10% restocking fee"""
    if is_refund:
        restocking_fee = float(original_amount) * 0.10
        final_refund = float(original_amount) - restocking_fee
        return final_refund, restocking_fee
    return float(original_amount), 0.0

calculate_refund_udf = udf(calculate_refund_with_fee, StructType([
    StructField("final_amount", DoubleType(), True),
    StructField("restocking_fee", DoubleType(), True)
]))

# Enhanced refund analysis
refund_analysis = fact_transactions.withColumn(
    "is_refund", col("transaction_type") == "REFUND"
).withColumn(
    "refund_calculation", calculate_refund_udf(col("total_amount"), col("is_refund"))
).select(
    "*",
    col("refund_calculation.final_amount").alias("calculated_final_amount"),
    col("refund_calculation.restocking_fee").alias("calculated_restocking_fee")
)

logger.info("Refund UDF with 10% restocking fee implemented")
refund_analysis.filter(col("transaction_type") == "REFUND").show(5)


INFO:__main__:✅ Refund UDF with 10% restocking fee implemented
INFO:py4j.clientserver:Received command c on object id p0


+------------+---------+------------+-------------+------------+------------+------+----------+----------------+--------------+----------+--------------+------------+---------+------------------+-----------------------+-------------------------+
|product_code|territory|customer_key|customer_name|order_number|total_amount|status|order_date|transaction_type|restocking_fee|net_amount|is_large_order|created_date|is_refund|refund_calculation|calculated_final_amount|calculated_restocking_fee|
+------------+---------+------------+-------------+------------+------------+------+----------+----------------+--------------+----------+--------------+------------+---------+------------------+-----------------------+-------------------------+
+------------+---------+------------+-------------+------------+------------+------+----------+----------------+--------------+----------+--------------+------------+---------+------------------+-----------------------+-------------------------+



In [0]:
# 9. Fiscal Date Aggregations using Calendar Dimension
fiscal_aggregations = fact_transactions.filter(col("transaction_type") == "SALE") \
    .join(calendar_df, date_format(fact_transactions.order_date, "yyyy-MM-dd") == calendar_df.CALENDAR_DATE, "left") \
    .groupBy("FISCAL_YEAR", "FISCAL_QUARTER", "FISCAL_MONTH_NAME") \
    .agg(
        sum("net_amount").alias("fiscal_sales"),
        count("order_number").alias("fiscal_orders"),
        avg("net_amount").alias("fiscal_avg_order")
    ).orderBy("FISCAL_YEAR", "FISCAL_QUARTER") \
    .withColumn("created_date", current_timestamp())

logger.info(f"Fiscal Aggregations: {fiscal_aggregations.count()} records")
fiscal_aggregations.show(10)


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:✅ Fiscal Aggregations: 1 records
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


+-----------+--------------+-----------------+------------+-------------+----------------+--------------------+
|FISCAL_YEAR|FISCAL_QUARTER|FISCAL_MONTH_NAME|fiscal_sales|fiscal_orders|fiscal_avg_order|        created_date|
+-----------+--------------+-----------------+------------+-------------+----------------+--------------------+
|       2025|             2|              Jun|    282300.0|         2823|           100.0|2025-06-21 21:34:...|
+-----------+--------------+-----------------+------------+-------------+----------------+--------------------+

